# Kaggle competition

## SLICED s01e10, Playoffs 2
## Predict outcomes of shelter animals 🥺


In [26]:
import pandas as pd
import numpy as np
import datetime


from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

from sklearn.ensemble import RandomForestClassifier

In [27]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


In [28]:
train = train.dropna()
train = train.reset_index(drop=True)

test = test.dropna()
test = test.reset_index(drop=True)

In [29]:
def preprocessing(df):
    df["date_of_birth"] = df["date_of_birth"].astype(dtype="datetime64") # Convert these two into datetime64 
    df["datetime"] = df["datetime"].astype(dtype="datetime64")
    
    df["age"] = None
    df['age'] = (df["datetime"] - df["date_of_birth"]).apply(lambda x:  x.days / 365.25) #Defining age as the diference between current time and dob.
        
    df=df.drop(['age_upon_outcome','date_of_birth','datetime','name','id'],axis=1) # we don't need these columns anymore.
    
    return df

In [30]:
train = preprocessing(train)
test = preprocessing(test)

In [31]:
train.columns

Index(['animal_type', 'breed', 'color', 'outcome_type', 'sex', 'spay_neuter',
       'age'],
      dtype='object')

In [32]:
encoder=ce.OneHotEncoder(cols=["animal_type","breed","color","sex","spay_neuter"],return_df=True, use_cat_names=True) # One hot encode these columns

In [33]:
train_features = train.drop("outcome_type",axis=1) # Seperate Y column
train_outcome = train["outcome_type"]
encoder.fit(train_features) # apply encoder on features
train_features = encoder.transform(train_features)

In [34]:
test = encoder.transform(test) # Use the same encoder on the test set

In [49]:
clf = RandomForestClassifier(max_depth=2, random_state=0) # Using Random Forest classifier

In [37]:
clf.fit(train_features, train_outcome)

RandomForestClassifier(max_depth=2, random_state=0)

In [39]:
predictions = clf.predict(test)
predictions_df =  pd.DataFrame(predictions, columns = ["outcome_type"])
encoder=ce.OneHotEncoder(cols="outcome_type",return_df=True, use_cat_names=True)

In [40]:
encoder.fit(train_outcome)
df_submission = encoder.transform(predictions_df)

In [50]:
df_submission

,adoption,no outcome,transfer
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
16179,1,0,0
16180,1,0,0
16181,1,0,0
16182,1,0,0


In [42]:
df_submission = df_submission.rename(columns={"outcome_type_adoption": "adoption", "outcome_type_transfer": "transfer", "outcome_type_no outcome": "no outcome"})

In [43]:
column_to_move = df_submission.pop("no outcome")
df_submission.insert(1, "no outcome", column_to_move)

In [44]:
df_submission.to_csv("submission.csv")

48552